## Import Dependencies

In [79]:
# Standard stuff
import numpy as np
import pandas as pd
import datetime as dt

# imports for data transformation and model
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# viz libraries
import matplotlib.pyplot as plt

# sql alchemy for connecting to database
from sqlalchemy import create_engine, func
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base


import os

## Clean Data

### Clean Artists Data

In [80]:
file_path = r'./artists.csv'
df_artists = pd.read_csv(file_path)
df_artists.head(10)

,id,followers,genres,name,popularity
0,0DheY5irMjBUeLybbCUEZ2,0.0,[],Armid & Amir Zare Pashai feat. Sara Rouzbehani,0
1,0DlhY15l3wsrnlfGio2bjU,5.0,[],ปูนา ภาวิณี,0
2,0DmRESX2JknGPQyO15yxg7,0.0,[],Sadaa,0
3,0DmhnbHjm1qw6NCYPeZNgJ,0.0,[],Tra'gruda,0
4,0Dn11fWM7vHQ3rinvWEl4E,2.0,[],Ioannis Panoutsopoulos,0
5,0DotfDlYMGqkbzfBhcA5r6,7.0,[],Astral Affect,0
6,0DqP3bOCiC48L8SM9gK4W8,1.0,[],Yung Seed,0
7,0Drs3maQb99iRglyTuxizI,0.0,[],Wi'Ma,0
8,0DsPeAi1gxPPnYjgpiEGSR,0.0,[],lentboy,0
9,0DtvnTxgZ9K5YaPS5jdlQW,20.0,[],addworks,0


In [81]:
df_artists.shape

(1104349, 5)

In [82]:
print(type(df_artists.loc[0,'genres']))

<class 'str'>


In [83]:
df_artists['genres'] = df_artists['genres'].str.strip('[]')

In [84]:
df_artists.head(10)

,id,followers,genres,name,popularity
0,0DheY5irMjBUeLybbCUEZ2,0.0,,Armid & Amir Zare Pashai feat. Sara Rouzbehani,0
1,0DlhY15l3wsrnlfGio2bjU,5.0,,ปูนา ภาวิณี,0
2,0DmRESX2JknGPQyO15yxg7,0.0,,Sadaa,0
3,0DmhnbHjm1qw6NCYPeZNgJ,0.0,,Tra'gruda,0
4,0Dn11fWM7vHQ3rinvWEl4E,2.0,,Ioannis Panoutsopoulos,0
5,0DotfDlYMGqkbzfBhcA5r6,7.0,,Astral Affect,0
6,0DqP3bOCiC48L8SM9gK4W8,1.0,,Yung Seed,0
7,0Drs3maQb99iRglyTuxizI,0.0,,Wi'Ma,0
8,0DsPeAi1gxPPnYjgpiEGSR,0.0,,lentboy,0
9,0DtvnTxgZ9K5YaPS5jdlQW,20.0,,addworks,0


In [85]:
nan_value = float('NaN')
df_artists.replace('', nan_value, inplace=True)
df_artists.dropna(subset=['genres'], inplace=True)

In [86]:
df_artists.reset_index(drop=True, inplace=True)
df_artists.head(10)

,id,followers,genres,name,popularity
0,0VLMVnVbJyJ4oyZs2L3Yl2,71.0,'carnaval cadiz',Las Viudas De Los Bisabuelos,6
1,0dt23bs4w8zx154C5xdVyl,63.0,'carnaval cadiz',Los De Capuchinos,5
2,0pGhoB99qpEJEsBQxgaskQ,64.0,'carnaval cadiz',Los “Pofesionales”,7
3,3HDrX2OtSuXLW5dLR85uN3,53.0,'carnaval cadiz',Los Que No Paran De Rajar,6
4,22mLrN5fkppmuUPsHx6i2G,59.0,"'classical harp', 'harp'",Vera Dulova,3
5,1OsJZxSshQD4BCg1VtwxsN,155.0,'classical contralto',Anna Larsson,19
6,5JRTWjFYThNR99D7IOEmn0,106.0,'british choir',BBC Chorus,16
7,4sTTEheJxmjwv9TmrHOaPz,288.0,"""children's story""",Honor Blackman,15
8,3zQdpHMTdJnV4aCzGqCBYK,3918.0,"'classic persian pop', 'persian traditional'",Javad Maroufi,26
9,7frYUe4C7A42uZqCzD34Y4,53636.0,"'desi pop', 'punjabi hip hop', 'punjabi pop'",Sultaan,53


In [87]:
df_artists.shape

(298616, 5)

In [88]:
#df.to_csv('Data/cleaned_artists.csv')

### Clean Tracks Data

In [89]:
df_tracks = pd.read_csv(r'.\tracks.csv')
print(df_tracks.shape)
df_tracks.head(10)

(586672, 20)


,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.744000,0.1510,0.1270,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.000000,0.1480,0.6550,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.021800,0.2120,0.4570,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.918000,0.1040,0.3970,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.130000,0.3110,0.1960,103.220,4
5,0BRXJHRNGQ3W4v9frnSfhu,Ave Maria,0,178933,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.227,0.2610,5,-12.343,1,0.0382,0.994,0.247000,0.0977,0.0539,118.891,4
6,0Dd9ImXtAtGwsmsAD69KZT,La Butte Rouge,0,134467,0,['Francis Marty'],['2nuMRGzeJ5jJEKlfS7rZ0W'],1922,0.510,0.3550,4,-12.833,1,0.1240,0.965,0.000000,0.1550,0.7270,85.754,5
7,0IA0Hju8CAgYfV1hwhidBH,La Java,0,161427,0,['Mistinguett'],['4AxgXfD7ISvJSTObqm4aIE'],1922,0.563,0.1840,4,-13.757,1,0.0512,0.993,0.000016,0.3250,0.6540,133.088,3
8,0IgI1UCz84pYeVetnl1lGP,Old Fashioned Girl,0,310073,0,['Greg Fieler'],['5nWlsH5RDgFuRAiDeOFVmf'],1922,0.488,0.4750,0,-16.222,0,0.0399,0.620,0.006450,0.1070,0.5440,139.952,4
9,0JV4iqw2lSKJaHBQZ0e5zK,Martín Fierro - Remasterizado,0,181173,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-29,0.548,0.0391,6,-23.228,1,0.1530,0.996,0.933000,0.1480,0.6120,75.595,3


In [90]:
df_tracks.dtypes

id                   object
name                 object
popularity            int64
duration_ms           int64
explicit              int64
artists              object
id_artists           object
release_date         object
danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
time_signature        int64
dtype: object

In [91]:
df_tracks['artists'] = df_tracks['artists'].str.strip("['']")
df_tracks.head(10)

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,Uli,['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.744000,0.1510,0.1270,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,Fernando Pessoa,['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.000000,0.1480,0.6550,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,Ignacio Corsini,['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.021800,0.2120,0.4570,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,Ignacio Corsini,['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.918000,0.1040,0.3970,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,Dick Haymes,['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.130000,0.3110,0.1960,103.220,4
5,0BRXJHRNGQ3W4v9frnSfhu,Ave Maria,0,178933,0,Dick Haymes,['3BiJGZsyX9sJchTqcSA7Su'],1922,0.227,0.2610,5,-12.343,1,0.0382,0.994,0.247000,0.0977,0.0539,118.891,4
6,0Dd9ImXtAtGwsmsAD69KZT,La Butte Rouge,0,134467,0,Francis Marty,['2nuMRGzeJ5jJEKlfS7rZ0W'],1922,0.510,0.3550,4,-12.833,1,0.1240,0.965,0.000000,0.1550,0.7270,85.754,5
7,0IA0Hju8CAgYfV1hwhidBH,La Java,0,161427,0,Mistinguett,['4AxgXfD7ISvJSTObqm4aIE'],1922,0.563,0.1840,4,-13.757,1,0.0512,0.993,0.000016,0.3250,0.6540,133.088,3
8,0IgI1UCz84pYeVetnl1lGP,Old Fashioned Girl,0,310073,0,Greg Fieler,['5nWlsH5RDgFuRAiDeOFVmf'],1922,0.488,0.4750,0,-16.222,0,0.0399,0.620,0.006450,0.1070,0.5440,139.952,4
9,0JV4iqw2lSKJaHBQZ0e5zK,Martín Fierro - Remasterizado,0,181173,0,Ignacio Corsini,['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-29,0.548,0.0391,6,-23.228,1,0.1530,0.996,0.933000,0.1480,0.6120,75.595,3


In [92]:
df_tracks['id_artists'] = df_tracks['id_artists'].str.strip("['']")
df_tracks.head(10)

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,Uli,45tIt06XoI0Iio4LBEVpls,1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.744000,0.1510,0.1270,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,Fernando Pessoa,14jtPCOoNZwquk5wd9DxrY,1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.000000,0.1480,0.6550,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,Ignacio Corsini,5LiOoJbxVSAMkBS2fUm3X2,1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.021800,0.2120,0.4570,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,Ignacio Corsini,5LiOoJbxVSAMkBS2fUm3X2,1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.918000,0.1040,0.3970,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,Dick Haymes,3BiJGZsyX9sJchTqcSA7Su,1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.130000,0.3110,0.1960,103.220,4
5,0BRXJHRNGQ3W4v9frnSfhu,Ave Maria,0,178933,0,Dick Haymes,3BiJGZsyX9sJchTqcSA7Su,1922,0.227,0.2610,5,-12.343,1,0.0382,0.994,0.247000,0.0977,0.0539,118.891,4
6,0Dd9ImXtAtGwsmsAD69KZT,La Butte Rouge,0,134467,0,Francis Marty,2nuMRGzeJ5jJEKlfS7rZ0W,1922,0.510,0.3550,4,-12.833,1,0.1240,0.965,0.000000,0.1550,0.7270,85.754,5
7,0IA0Hju8CAgYfV1hwhidBH,La Java,0,161427,0,Mistinguett,4AxgXfD7ISvJSTObqm4aIE,1922,0.563,0.1840,4,-13.757,1,0.0512,0.993,0.000016,0.3250,0.6540,133.088,3
8,0IgI1UCz84pYeVetnl1lGP,Old Fashioned Girl,0,310073,0,Greg Fieler,5nWlsH5RDgFuRAiDeOFVmf,1922,0.488,0.4750,0,-16.222,0,0.0399,0.620,0.006450,0.1070,0.5440,139.952,4
9,0JV4iqw2lSKJaHBQZ0e5zK,Martín Fierro - Remasterizado,0,181173,0,Ignacio Corsini,5LiOoJbxVSAMkBS2fUm3X2,1922-03-29,0.548,0.0391,6,-23.228,1,0.1530,0.996,0.933000,0.1480,0.6120,75.595,3


In [93]:
df_tracks.isnull().sum()

id                   0
name                71
popularity           0
duration_ms          0
explicit             0
artists              0
id_artists           0
release_date         0
danceability         0
energy               0
key                  0
loudness             0
mode                 0
speechiness          0
acousticness         0
instrumentalness     0
liveness             0
valence              0
tempo                0
time_signature       0
dtype: int64

In [94]:
df_tracks.dropna()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,Uli,45tIt06XoI0Iio4LBEVpls,1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.744000,0.1510,0.1270,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,Fernando Pessoa,14jtPCOoNZwquk5wd9DxrY,1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.000000,0.1480,0.6550,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,Ignacio Corsini,5LiOoJbxVSAMkBS2fUm3X2,1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.021800,0.2120,0.4570,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,Ignacio Corsini,5LiOoJbxVSAMkBS2fUm3X2,1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.918000,0.1040,0.3970,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,Dick Haymes,3BiJGZsyX9sJchTqcSA7Su,1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.130000,0.3110,0.1960,103.220,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586667,5rgu12WBIHQtvej2MdHSH0,云与海,50,258267,0,阿YueYue,1QLBXKM5GCpyQQSVMNZqrZ,2020-09-26,0.560,0.5180,0,-7.471,0,0.0292,0.785,0.000000,0.0648,0.2110,131.896,4
586668,0NuWgxEp51CutD2pJoF4OM,blind,72,153293,0,ROLE MODEL,1dy5WNgIKQU6ezkpZs4y8z,2020-10-21,0.765,0.6630,0,-5.223,1,0.0652,0.141,0.000297,0.0924,0.6860,150.091,4
586669,27Y1N4Q4U3EfDU5Ubw8ws2,What They'll Say About Us,70,187601,0,FINNEAS,37M5pPGs6V1fchFJSgCguX,2020-09-02,0.535,0.3140,7,-12.823,0,0.0408,0.895,0.000150,0.0874,0.0663,145.095,4
586670,45XJsGpFTyzbzeWK8VzR8S,A Day At A Time,58,142003,0,"Gentle Bones', 'Clara Benin","4jGPdu95icCKVF31CcFKbS', '5ebPSE9YI5aLeZ1Z2gkqjn",2021-03-05,0.696,0.6150,10,-6.212,1,0.0345,0.206,0.000003,0.3050,0.4380,90.029,4


In [95]:
featured_index = df_tracks[df_tracks['artists'].str.contains(',')].index


In [96]:
featured_index = df_tracks[df_tracks['artists'].str.contains(',')].index
multiple_artists = df_tracks[df_tracks['artists'].str.contains(',')]
first_artists = multiple_artists['artists'].str.extract(r'((\w*\s)*\w*)\'\,')[0]
df_tracks.loc[df_tracks['artists'].str.contains(','),'artists'] = first_artists
# df_tracks['artists'] = df_tracks['artists'].fillna(value=first_artists)
df_tracks.loc[featured_index,:]

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
36,1MD0Obbza9l0t0Zpgcwagy,And Mimi,0,186147,0,Dick Haymes,"3BiJGZsyX9sJchTqcSA7Su', '58wzyK6DupVsypvs3QV2...",1922,0.284,0.223,9,-15.733,0,0.0453,0.980000,0.081300,0.3450,0.2260,77.234,3
37,1O9iZyzufN1fUdVO97mmm5,How High the Moon,0,175333,0,Dick Haymes,"3BiJGZsyX9sJchTqcSA7Su', '5MpELOfAiq7aIBTij30p...",1922,0.328,0.307,0,-11.573,1,0.0484,0.989000,0.277000,0.2100,0.2590,117.225,4
84,3h8ioTTWfrC25hrHwQQpLc,All Or Nothing at All,0,164320,0,Dick Haymes,"3BiJGZsyX9sJchTqcSA7Su', '5MpELOfAiq7aIBTij30p...",1922,0.237,0.555,2,-6.611,0,0.0401,0.812000,0.000262,0.3660,0.2920,92.213,3
96,4DE7VBCgyTjqV0Plc9Ra0U,Hush-A-Bye (Wee Rose of Kilarney),0,156880,0,Dick Haymes,"3BiJGZsyX9sJchTqcSA7Su', '3HqN7Sq7rmpOEI9UV5ER...",1922,0.279,0.128,0,-19.793,1,0.0406,0.994000,0.613000,0.1160,0.1450,79.121,5
130,6GhbZRE9N2TPf86zI1dIxQ,You'll Never Know,0,165253,0,Dick Haymes,"3BiJGZsyX9sJchTqcSA7Su', '5o8E07TcYqIefZpDejymAd",1922,0.392,0.128,2,-15.220,1,0.0378,0.994000,0.153000,0.0983,0.2260,126.728,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586638,3wGIJgoBmapSwFjafNJQ6X,Allies (FYHYM2019),1,55651,0,Davey Asprey,"7luLRXGBygBN8vUO8T9uHf', '6Gs2jNsD9XkEYUJZOSx4qk",2020-01-05,0.462,0.953,1,-7.582,0,0.1190,0.003440,0.934000,0.5960,0.3470,137.984,5
586642,44tvGaqisGSy4Qy45vd0CF,Thousand Pieces (FYHYM2019),1,83477,0,Z,"22y1A8uijPBYxDPOrMWZPb', '78JtIs2G2CpfOZadEs1s...",2020-01-05,0.393,0.997,1,-6.145,0,0.1450,0.000239,0.002680,0.7560,0.0639,137.938,4
586643,1932nBdtEgMpDGkpl93cS4,Saving Angel (FYHYM2019),1,125216,0,RAM,"4f0a5IgkYFHFts5Z9N9SDX', '51GkQKgac6wqdicVA2DvPu",2020-01-05,0.458,0.997,6,-6.124,0,0.1940,0.000898,0.872000,0.4600,0.0393,137.996,4
586663,4ow9HehIdFii1cggylW2k0,四季予你 - DJ版,47,156393,0,程響,"7nKA1c1Qn6nI0XA8yburf3', '7g8hOWXtGS16J30CMU1SR7",2020-12-29,0.677,0.970,0,-3.388,0,0.0446,0.134000,0.002340,0.3020,0.9080,140.026,4


In [102]:
# do the same for artists id 
multiple_id = df_tracks[df_tracks['id_artists'].str.contains(',')]
first_id = multiple_id['id_artists'].str.extract(r'(\w*)\'')[0]
df_tracks.loc[df_tracks['id_artists'].str.contains(','),'id_artists'] = first_id
df_tracks.loc[featured_index,:]

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
36,1MD0Obbza9l0t0Zpgcwagy,And Mimi,0,186147,0,Dick Haymes,3BiJGZsyX9sJchTqcSA7Su,1922,0.284,0.223,9,-15.733,0,0.0453,0.980000,0.081300,0.3450,0.2260,77.234,3
37,1O9iZyzufN1fUdVO97mmm5,How High the Moon,0,175333,0,Dick Haymes,3BiJGZsyX9sJchTqcSA7Su,1922,0.328,0.307,0,-11.573,1,0.0484,0.989000,0.277000,0.2100,0.2590,117.225,4
84,3h8ioTTWfrC25hrHwQQpLc,All Or Nothing at All,0,164320,0,Dick Haymes,3BiJGZsyX9sJchTqcSA7Su,1922,0.237,0.555,2,-6.611,0,0.0401,0.812000,0.000262,0.3660,0.2920,92.213,3
96,4DE7VBCgyTjqV0Plc9Ra0U,Hush-A-Bye (Wee Rose of Kilarney),0,156880,0,Dick Haymes,3BiJGZsyX9sJchTqcSA7Su,1922,0.279,0.128,0,-19.793,1,0.0406,0.994000,0.613000,0.1160,0.1450,79.121,5
130,6GhbZRE9N2TPf86zI1dIxQ,You'll Never Know,0,165253,0,Dick Haymes,3BiJGZsyX9sJchTqcSA7Su,1922,0.392,0.128,2,-15.220,1,0.0378,0.994000,0.153000,0.0983,0.2260,126.728,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586638,3wGIJgoBmapSwFjafNJQ6X,Allies (FYHYM2019),1,55651,0,Davey Asprey,7luLRXGBygBN8vUO8T9uHf,2020-01-05,0.462,0.953,1,-7.582,0,0.1190,0.003440,0.934000,0.5960,0.3470,137.984,5
586642,44tvGaqisGSy4Qy45vd0CF,Thousand Pieces (FYHYM2019),1,83477,0,Z,22y1A8uijPBYxDPOrMWZPb,2020-01-05,0.393,0.997,1,-6.145,0,0.1450,0.000239,0.002680,0.7560,0.0639,137.938,4
586643,1932nBdtEgMpDGkpl93cS4,Saving Angel (FYHYM2019),1,125216,0,RAM,4f0a5IgkYFHFts5Z9N9SDX,2020-01-05,0.458,0.997,6,-6.124,0,0.1940,0.000898,0.872000,0.4600,0.0393,137.996,4
586663,4ow9HehIdFii1cggylW2k0,四季予你 - DJ版,47,156393,0,程響,7nKA1c1Qn6nI0XA8yburf3,2020-12-29,0.677,0.970,0,-3.388,0,0.0446,0.134000,0.002340,0.3020,0.9080,140.026,4


In [74]:
df_tracks.drop(featured_index, inplace=True)
print(df_tracks.shape)
df_tracks.head(10)

(478571, 20)


,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,Uli,45tIt06XoI0Iio4LBEVpls,1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.744000,0.1510,0.1270,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,Fernando Pessoa,14jtPCOoNZwquk5wd9DxrY,1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.000000,0.1480,0.6550,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,Ignacio Corsini,5LiOoJbxVSAMkBS2fUm3X2,1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.021800,0.2120,0.4570,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,Ignacio Corsini,5LiOoJbxVSAMkBS2fUm3X2,1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.918000,0.1040,0.3970,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,Dick Haymes,3BiJGZsyX9sJchTqcSA7Su,1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.130000,0.3110,0.1960,103.220,4
5,0BRXJHRNGQ3W4v9frnSfhu,Ave Maria,0,178933,0,Dick Haymes,3BiJGZsyX9sJchTqcSA7Su,1922,0.227,0.2610,5,-12.343,1,0.0382,0.994,0.247000,0.0977,0.0539,118.891,4
6,0Dd9ImXtAtGwsmsAD69KZT,La Butte Rouge,0,134467,0,Francis Marty,2nuMRGzeJ5jJEKlfS7rZ0W,1922,0.510,0.3550,4,-12.833,1,0.1240,0.965,0.000000,0.1550,0.7270,85.754,5
7,0IA0Hju8CAgYfV1hwhidBH,La Java,0,161427,0,Mistinguett,4AxgXfD7ISvJSTObqm4aIE,1922,0.563,0.1840,4,-13.757,1,0.0512,0.993,0.000016,0.3250,0.6540,133.088,3
8,0IgI1UCz84pYeVetnl1lGP,Old Fashioned Girl,0,310073,0,Greg Fieler,5nWlsH5RDgFuRAiDeOFVmf,1922,0.488,0.4750,0,-16.222,0,0.0399,0.620,0.006450,0.1070,0.5440,139.952,4
9,0JV4iqw2lSKJaHBQZ0e5zK,Martín Fierro - Remasterizado,0,181173,0,Ignacio Corsini,5LiOoJbxVSAMkBS2fUm3X2,1922-03-29,0.548,0.0391,6,-23.228,1,0.1530,0.996,0.933000,0.1480,0.6120,75.595,3
